In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
dataBook=pd.read_csv('/Users/sandrine//Desktop/Prework/Projects_Ironhack/Book_loan/docprets.csv',sep=';')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Objectif : Analyse Book loan trends of Parisian people and predict number of Loan per books

In [ ]:
""" Loan may depend on:
Book "Age"(accessible via Year column), 
the librairie and its stock,
the author,
the publisher (column Editeur)
the format

In [3]:
dataBook.head()

,N° de notice,ISBN,ISSN,EAN,Langue,Titre,Edition,Editeur,Date,Format,...,Vaclav Havel,Valeyre,Vandamme,Vaugirard,Auteur,Co-auteur,Auteur secondaire,Collectivité auteur,Collectivité auteur secondaire,Collectivité co-auteur
0,176870,NaN,NaN,NaN,français,La Politique étrangère en France | Texte impri...,NaN,Documentation française ;Ministère des affaire...,2003,386 p. | 28 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRANCE. Direction de la documentation française,France. Ministère des relations extérieures;Di...,NaN
1,176733,2-8435-2107-6,NaN,NaN,français,Voyages en Sicile | Texte imprimé | Dominique ...,NaN,"Diderot éditeur, arts et sciences",1998,1 vol. (299 p.) | 18 cm,...,NaN,NaN,NaN,NaN,"Denon, Dominique Vivant",NaN,NaN,NaN,NaN,NaN
2,176766,2-259-00278-1,NaN,NaN,français,0Voyages | [2] | Russie | Texte imprimé | Niko...,NaN,Plon,1977,322 p.-[1] f. de pl. | 20 cm,...,NaN,NaN,NaN,NaN,"Kazantzákīs, Níkos",NaN,"Princet, Liliane",NaN,NaN,NaN
3,176779,NaN,NaN,NaN,français,L'harmonica blues pour débutants | Texte imprimé,NaN,Emf,1997,1 méthode musicale (62 p.) | 30 cm | éventuell...,...,NaN,NaN,NaN,NaN,"Kinsella, Mick",NaN,NaN,NaN,NaN,NaN
4,176892,0-06-443063-4,NaN,NaN,anglais,A letter to Amy | Texte imprimé | Ezra Jack Keats,NaN,Harper Trophy,1984,30 p. | ill. en coul. | 21 cm,...,NaN,NaN,NaN,NaN,"Keats, Ezra Jack",NaN,NaN,NaN,NaN,NaN


In [4]:
#Check general info from DataFrame
dataBook.shape,dataBook.dtypes

((815534, 103),
 N° de notice                        int64
 ISBN                               object
 ISSN                               object
 EAN                                object
 Langue                             object
                                     ...  
 Co-auteur                          object
 Auteur secondaire                  object
 Collectivité auteur                object
 Collectivité auteur secondaire     object
 Collectivité co-auteur             object
 Length: 103, dtype: object)

In [5]:
#Replace ' ' by '_in column names'
dataBook.columns=dataBook.columns.str.replace(' ','_')
dataBook.columns.to_list()

['N°_de_notice',
 'ISBN',
 'ISSN',
 'EAN',
 'Langue',
 'Titre',
 'Edition',
 'Editeur',
 'Date',
 'Format',
 'Collection',
 'Titre_de_série',
 'N°',
 'Auteur_Nom',
 'Auteur_Prénom',
 'Auteur_Dates',
 'Co-auteur_Nom',
 'Co-auteur_Prénom',
 'Dates',
 'Auteur_secondaire_Nom',
 'Auteur_secondaire_Prénom',
 'Auteur_secondaire_Dates',
 'Auteur_collectivité',
 'Subdivision_auteur_collectivité',
 'Co-auteur_collectivité',
 'Subdivision_co-auteur_collectivité',
 'Auteur_secondaire_collectivité',
 'Subdivision_auteur_secondaire_collectivité',
 'Indice',
 'Cote_majoritaire',
 'Type_de_document',
 'Nombre_de_localisations',
 'Nombre_de_prêt_total',
 'Nombre_de_prêts_2017',
 'Nombre_de_prêt_année_2018_au_26_juillet_2018',
 "Nombre_d'exemplaires",
 'Catégorie_statistique_1',
 'Catégorie_statistique_2',
 'Aimé_Césaire',
 'Amélie',
 'André_Malraux',
 'Andrée_Chedid',
 'Arthur_Rimbaud',
 'Assia_Djebar',
 'Batignolles',
 'Benjamin_Rabier',
 'Buffon',
 'Canopée_-_la_fontaine',
 'Chaptal',
 'Charlotte_Del

In [6]:
# Prepare list of columns to drop due to missing values
to_drop=[]
to_drop.extend(dataBook.iloc[:,100:103].columns.to_list())
#Prepare list of columns for which we want to get dummies
Dummies_NA=[]


# NULL VALUES TREATMENT

In [7]:
#Check Number of Null values per columns
dataBook.isna().sum().to_dict()

{'N°_de_notice': 0,
 'ISBN': 323243,
 'ISSN': 813046,
 'EAN': 416532,
 'Langue': 81024,
 'Titre': 0,
 'Edition': 753989,
 'Editeur': 5012,
 'Date': 4727,
 'Format': 9126,
 'Collection': 452201,
 'Titre_de_série': 731570,
 'N°': 747164,
 'Auteur_Nom': 195970,
 'Auteur_Prénom': 226043,
 'Auteur_Dates': 479400,
 'Co-auteur_Nom': 742303,
 'Co-auteur_Prénom': 745706,
 'Dates': 780798,
 'Auteur_secondaire_Nom': 434722,
 'Auteur_secondaire_Prénom': 445681,
 'Auteur_secondaire_Dates': 643727,
 'Auteur_collectivité': 778776,
 'Subdivision_auteur_collectivité': 814619,
 'Co-auteur_collectivité': 812310,
 'Subdivision_co-auteur_collectivité': 815398,
 'Auteur_secondaire_collectivité': 734528,
 'Subdivision_auteur_secondaire_collectivité': 812303,
 'Indice': 382046,
 'Cote_majoritaire': 3,
 'Type_de_document': 2,
 'Nombre_de_localisations': 1,
 'Nombre_de_prêt_total': 36674,
 'Nombre_de_prêts_2017': 3,
 'Nombre_de_prêt_année_2018_au_26_juillet_2018': 44298,
 "Nombre_d'exemplaires": 3,
 'Catégorie_

In [8]:
#dataBook["Auteur_Dates"].value_counts()
#Column not necessary for the model so I drop it.
to_drop.append('Auteur_Dates')

In [9]:
#check whether I can drop or replace 'Auteur', 'Auteur Prenom' and 'Auteur Nom' columns
dftest=dataBook[(dataBook['Auteur_Nom']+', '+dataBook['Auteur_Prénom'])!=dataBook['Auteur']]
df=dftest[(dftest['Auteur_Nom']==dftest['Auteur'])|(dftest['Auteur_Prénom']==dftest['Auteur'])]
df[['Auteur_Nom','Auteur_Prénom','Auteur']]

df1=dftest[(dftest['Auteur_Nom']!=dftest['Auteur'])&(dftest['Auteur_Prénom']!=dftest['Auteur'])]
df1[['Auteur_Nom','Auteur_Prénom','Auteur']]
df1['Auteur_Prénom'].isna().sum()==df1['Auteur_Nom'].isna().sum()==df1['Auteur'].isna().sum()
#Author column is a mix of columns 'Auteur_Nom' and 'Auteur_Prénom' . However the format of Author does not
#suit me.


True

In [43]:
# I will create a  new column for Author.
dataBook['Author']=dataBook['Auteur_Prénom']+'|'+dataBook['Auteur_Nom']
dataBook['Author'].fillna('UnknownAuthor',inplace=True)
dataBook['Author']=dataBook['Author'].map(lambda x : merge_authors(x))
#dataBook[dataBook['Author'].isna()]


In [11]:
to_drop.extend(['Auteur_Prénom','Auteur_Nom','Auteur'])

In [13]:
#check whether I can drop or replace 'Auteur_secondaire_Nom', 'Auteur_secondaire_Prénom' 
#and 'Auteur_secondaire' columns. Auteur secondaire include translaters.
#df3=dataBook[(dataBook['Auteur_secondaire_Nom']+', '+dataBook['Auteur_secondaire_Prénom'])!=dataBook['Auteur_secondaire']]
#df3[['Auteur_secondaire_Nom','Auteur_secondaire_Prénom','Auteur_secondaire']]
# same conclusions as for Author.

In [40]:
def merge_authors(x):
    if ';' in x:
        y=x.split('|')
        z=[i for i in zip(y[0].split(';'),y[1].split(';'))]
        w=[' '.join(i) for i in z]
        return w
    else:
        w=x.replace("|", " ")
        return w
    
authors="Watson;Stephen;Padmore;Varcoe;Hickox,|Janice;Pamela Helen;Mark;Stephen;Richard"
test='Liliane|Princet'
y=merge_authors(test)
y

'Liliane Princet'

In [45]:
# I will create a  new column for Secondary Author.

dataBook['Author_secondary']=dataBook['Auteur_secondaire_Prénom']+'|'+dataBook['Auteur_secondaire_Nom']
dataBook['Author_secondary'].fillna('UnknownAuthor',inplace=True)
dataBook['Author_secondary']=dataBook['Author_secondary'].map(lambda x : merge_authors(x))

In [16]:
to_drop.extend(['Auteur_secondaire_Prénom','Auteur_secondaire_Nom','Auteur_secondaire'])

In [20]:
#dataBook["Co-auteur_Nom"].value_counts()

In [47]:
# I will create a  new column for Co Author Author.

dataBook['Co_Author']=dataBook['Co-auteur_Prénom']+'|'+dataBook['Co-auteur_Nom']
dataBook['Co_Author'].fillna('UnknownAuthor',inplace=True)
dataBook['Co_Author']=dataBook['Co_Author'].map(lambda x : merge_authors(x))

In [49]:
#dataBook[dataBook['N°_de_notice']==176958].to_dict()

In [53]:
#dataBook["Auteur_secondaire_Dates"].value_counts()
to_drop.append('Auteur_secondaire_Dates')

In [ ]:
# Avoir si je droppeou si cela a un intérêt plus tard.

#to_drop2=(['Auteur_collectivité','Subdivision_auteur_collectivité','Co-auteur_collectivité','Subdivision_co-auteur_collectivité','Auteur_secondaire_collectivité','Subdivision_auteur_secondaire_collectivité'])

"""'Auteur_collectivité'])
 'Subdivision_auteur_collectivité': 814619,
 'Co-auteur_collectivité': 812310,
 'Subdivision_co-auteur_collectivité': 815398,
 'Auteur_secondaire_collectivité': 734528,
 'Subdivision_auteur_secondaire_collectivité': 812303,"""

In [62]:
#dataBook["Subdivision_auteur_secondaire_collectivité"].value_counts()

In [72]:
#dataBook["Indice"].value_counts()

In [73]:
#dataBook[dataBook["Indice"]=='914.3;320.943;330.943']

In [74]:
to_drop.append('Indice')

In [75]:
dataBook["Cote_majoritaire"].value_counts()

'PER                     1291
'CHA                     1018
'MAR                      902
'MAC                      862
'BER                      705
                         ... 
'MAGASIN N 48115            1
'BD JOUR J T.1              1
'750.9 LET T.1              1
'F. ITALIEN 305.8 ITA       1
'BLU-RAY 2.4 ZAP            1
Name: Cote_majoritaire, Length: 350077, dtype: int64

In [ ]:
Cote_majoritaire

In [69]:
dataBook.iloc[8].to_dict()

{'N°_de_notice': 177027,
 'ISBN': '2-7028-7143-7',
 'ISSN': nan,
 'EAN': nan,
 'Langue': 'français',
 'Titre': "Ice limit | Texte imprimé | Douglas Preston & Lincoln Child | trad. de l'américain par Karine Laléchère",
 'Edition': nan,
 'Editeur': 'le Grand livre du mois',
 'Date': '2002',
 'Format': '455 p. | carte, couv. ill. en coul. | 24 cm',
 'Collection': nan,
 'Titre_de_série': nan,
 'N°': nan,
 'Auteur_Nom': 'Preston',
 'Auteur_Prénom': 'Douglas',
 'Auteur_Dates': '1956-....',
 'Co-auteur_Nom': 'Child',
 'Co-auteur_Prénom': 'Lincoln',
 'Dates': '1957-....',
 'Auteur_secondaire_Nom': 'Lalechère',
 'Auteur_secondaire_Prénom': 'Karine',
 'Auteur_secondaire_Dates': '1967-....',
 'Auteur_collectivité': nan,
 'Subdivision_auteur_collectivité': nan,
 'Co-auteur_collectivité': nan,
 'Subdivision_co-auteur_collectivité': nan,
 'Auteur_secondaire_collectivité': nan,
 'Subdivision_auteur_secondaire_collectivité': nan,
 'Indice': '813',
 'Cote_majoritaire': "'PRE",
 'Type_de_document': 'Liv

In [50]:
dataBook.head(20)

,N°_de_notice,ISBN,ISSN,EAN,Langue,Titre,Edition,Editeur,Date,Format,...,Vaugirard,Auteur,Co-auteur,Auteur_secondaire,Collectivité_auteur,Collectivité_auteur_secondaire_,Collectivité_co-auteur_,Author,Author_secondary,Co_Author
0,176870,NaN,NaN,NaN,français,La Politique étrangère en France | Texte impri...,NaN,Documentation française ;Ministère des affaire...,2003,386 p. | 28 cm,...,0.0,NaN,NaN,NaN,FRANCE. Direction de la documentation française,France. Ministère des relations extérieures;Di...,NaN,UnknownAuthor,UnknownAuthor,UnknownAuthor
1,176733,2-8435-2107-6,NaN,NaN,français,Voyages en Sicile | Texte imprimé | Dominique ...,NaN,"Diderot éditeur, arts et sciences",1998,1 vol. (299 p.) | 18 cm,...,0.0,"Denon, Dominique Vivant",NaN,NaN,NaN,NaN,NaN,Dominique Vivant Denon,UnknownAuthor,UnknownAuthor
2,176766,2-259-00278-1,NaN,NaN,français,0Voyages | [2] | Russie | Texte imprimé | Niko...,NaN,Plon,1977,322 p.-[1] f. de pl. | 20 cm,...,0.0,"Kazantzákīs, Níkos",NaN,"Princet, Liliane",NaN,NaN,NaN,Níkos Kazantzákīs,Liliane Princet,UnknownAuthor
3,176779,NaN,NaN,NaN,français,L'harmonica blues pour débutants | Texte imprimé,NaN,Emf,1997,1 méthode musicale (62 p.) | 30 cm | éventuell...,...,0.0,"Kinsella, Mick",NaN,NaN,NaN,NaN,NaN,Mick Kinsella,UnknownAuthor,UnknownAuthor
4,176892,0-06-443063-4,NaN,NaN,anglais,A letter to Amy | Texte imprimé | Ezra Jack Keats,NaN,Harper Trophy,1984,30 p. | ill. en coul. | 21 cm,...,0.0,"Keats, Ezra Jack",NaN,NaN,NaN,NaN,NaN,Ezra Jack Keats,UnknownAuthor,UnknownAuthor
5,176932,978-2226-15-7881;2-226-15788-3,NaN,NaN,français,Un camping-car jaune fromage | Texte imprimé |...,NaN,Albin Michel Jeunesse,2005,117 p. | ill. en coul. | 19 cm,...,0.0,"Stilton, Geronimo",NaN,NaN,NaN,NaN,NaN,Geronimo Stilton,UnknownAuthor,UnknownAuthor
6,176954,2-7440-8067-5,NaN,NaN,français,Le manuel complet de typographie | Texte impri...,NaN,Peachpit press,cop. 2003,"XXI-324 p. | ill., couv. ill. | 30 cm",...,0.0,"Felici, James",NaN,NaN,NaN,NaN,NaN,James Felici,UnknownAuthor,UnknownAuthor
7,176958,NaN,NaN,NaN,latin,Theresienmesse - Kleine Orgelmesse | Enregistr...,NaN,Chandos,1996,1 compact | 1 brochure,...,0.0,"Haydn, Joseph",NaN,"Watson;Stephen;Padmore;Varcoe;Hickox, Janice;P...",NaN,Collegium Musicum 90,NaN,Joseph Haydn,"[Janice Watson, Pamela Helen Stephen, Mark Pad...",UnknownAuthor
8,177027,2-7028-7143-7,NaN,NaN,français,Ice limit | Texte imprimé | Douglas Preston & ...,NaN,le Grand livre du mois,2002,"455 p. | carte, couv. ill. en coul. | 24 cm",...,0.0,"Preston, Douglas","Child, Lincoln","Lalechère, Karine",NaN,NaN,NaN,Douglas Preston,Karine Lalechère,Lincoln Child
9,177030,NaN,NaN,NaN,NaN,"Touché | Enregistrement sonore | K. Wheeler, P...",NaN,Justin Time,1996,1 compact | 1 dépliant,...,0.0,NaN,NaN,"Wheeler;Bley, Kenny;Paul",NaN,NaN,NaN,UnknownAuthor,"[Kenny Wheeler, Paul Bley]",UnknownAuthor


In [ ]:
"""Columns nb 38 to 97 are the stock of book available for each librairies 
Null values will be replaced by 0"""

In [17]:
# Replace null values in Librairies colums by 0 since Nan<=>'book title not in this librairy'
dataBook.iloc[:,38:97]=dataBook.iloc[:,38:97].fillna(0)

In [ ]:
#dataBook["Nombre_d'exemplaires"].value_counts()

In [ ]:
#dataBook[dataBook["Nombre_d'exemplaires"]==1149.0].T.to_dict()

In [ ]:
#dataBook[dataBook['Nombre_de_localisations']==57.0].T.to_dict()

In [ ]:
#dataBook['ISBN'].value_counts() # Should not be use for the moment except for looking for info

In [ ]:
#dataBook[dataBook['ISBN']=='9787530450123'] # Same ISBN for al the "tomes" ex: Encyclopedie

In [ ]:
#Look at the diffrent types/format of info in Date columns
# Date give us more or less the age of the book (printing, edition,)
dataBook['Date'].value_counts().to_dict()

In [ ]:
# Format of Date are pretty different. Some values are not complete or not data
#Action:correct date to Year only
[re.findall(r'.*\d{4}.*',i) for i in list(dataBook['Date'].values)]



In [ ]:
list(dataBook['Date'].values)

In [ ]:
list('Auto,mobile')

In [ ]:
x='Auto,mobile'
y=x.split(',')

In [ ]:
type(y)